In [30]:
%pip install pandas numpy transformers matplotlib opencv-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pytorch (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "/home/akos/Documents/GitHub/experiments/handwritten/segmentation_with_tokens/.venv/lib/python3.7/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/home/akos/Documents/GitHub/experiments/handwritten/segmentation_with_tokens/.venv/lib/python3.7/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/home/akos/Documents/GitHub/experiments/handwritten/segmentation_with_tokens/.venv/lib/python3.7/site-packages/pip/_vendor/

In [32]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 7.3 MB/s eta 0:00:00:00:01m00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 41.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 10.6 MB/s eta 0:00:0000:0100:01m


In [71]:
!pip install nltk

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 32.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [6]:
!git clone https://huggingface.co/microsoft/trocr-large-handwritten

fatal: destination path 'trocr-large-handwritten' already exists and is not an empty directory.


In [ ]:
#wget csak linuxon műkszik
!wget "https://huggingface.co/datasets/zhixiaoni/CROHME_selected_Train_2014_png/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet"

import cv2
import matplotlib.pyplot as plt
i

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')
i_list = [0]

def OCR(row,i_list,processor,model):
    i_list[0]= i_list[0]+1
    print(i_list[0])
    nparr = np.fromstring(row["image"]["bytes"], np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    #plt.figure(figsize=(10,10)); plt.axis('off'); plt.imshow(img)
    #plt.show()
    pixel_values = processor(images=img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
    return generated_text

In [10]:
df = pd.read_parquet("./0000.parquet")
df

,image
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
...,...
5613,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
5614,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
5615,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
5616,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...


In [ ]:
df = df.head(1000)
df["text"] = df.apply(lambda x: OCR(x, i_list,processor,model), axis=1)

1


<ipython-input-13-cbf4a8af245b>:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  nparr = np.fromstring(row["image"]["bytes"], np.uint8)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


s - ( E.O. ( (m-2 ) " ) #
2


In [ ]:
df.shape[0]

100

In [ ]:
df.to_csv('ocr_results.csv')

In [7]:
df = pd.read_csv("ocr_results.csv")

In [23]:
def get_patterns(ocr_results=['alma', 'korte', 'balma', 'almkortelma'], pattern_len=3):
    patterns = []
    for res in ocr_results:
        word_len = len(res)
        start_index = 0
        print(f"checking patterns with: {res}")
        for i in range(0, word_len-1):
            end_index = i+pattern_len
            possible_pattern = res[start_index : end_index]
            if (len(possible_pattern) != pattern_len):
                break
            print(f"\tpossible_patter: {possible_pattern}, indexes: {start_index}:{end_index}")

            #compare with other results
            for other_result in ocr_results:
                if (possible_pattern in other_result and other_result != res):
                    patterns.append(possible_pattern)
            start_index += 1
        print()
    print(f"results: {patterns}")
    return patterns

def filter_patterns(patterns, min_occurance=3):
    pattern_ocs = []
    values = {}
    temp_arr = []


    for pattern in patterns:
        print(f"index: {patterns.index(pattern)}, from : {len(patterns)}")
        if (pattern not in temp_arr):
            temp_arr.append(pattern)
            counter = -1
            for other in patterns:
                if (other == pattern):
                    counter += 1
            if (counter >= min_occurance):
                values = {}
                values["pattern"] = pattern
                values["counter"] = counter
                pattern_ocs.append(values)
    return pattern_ocs

In [24]:
possible_patterns = get_patterns(ocr_results=df['text'].to_list(), pattern_len=4)

checking patterns with: s - ( E.O. ( (m-2 ) " ) #
	possible_patter: s - , indexes: 0:4
	possible_patter:  - (, indexes: 1:5
	possible_patter: - ( , indexes: 2:6
	possible_patter:  ( E, indexes: 3:7
	possible_patter: ( E., indexes: 4:8
	possible_patter:  E.O, indexes: 5:9
	possible_patter: E.O., indexes: 6:10
	possible_patter: .O. , indexes: 7:11
	possible_patter: O. (, indexes: 8:12
	possible_patter: . ( , indexes: 9:13
	possible_patter:  ( (, indexes: 10:14
	possible_patter: ( (m, indexes: 11:15
	possible_patter:  (m-, indexes: 12:16
	possible_patter: (m-2, indexes: 13:17
	possible_patter: m-2 , indexes: 14:18
	possible_patter: -2 ), indexes: 15:19
	possible_patter: 2 ) , indexes: 16:20
	possible_patter:  ) ", indexes: 17:21
	possible_patter: ) " , indexes: 18:22
	possible_patter:  " ), indexes: 19:23
	possible_patter: " ) , indexes: 20:24
	possible_patter:  ) #, indexes: 21:25

checking patterns with: s ( Es-on-sym ).
	possible_patter: s ( , indexes: 0:4
	possible_patter:  ( E, index

In [25]:
patterns = filter_patterns(patterns=possible_patterns, min_occurance=5)
#print(patterns)
pattern_df = pd.DataFrame.from_records(patterns)
pattern_df = pattern_df.drop_duplicates()
pattern_df.to_csv("patterns.csv", index=False)

pattern_df

index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 0, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 10, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 23, from : 56227
index: 38, from : 56227
index: 38, from : 56227
index: 38, from : 56227
index: 38, from : 56227
in

,pattern,counter
0,s -,119
1,- (,207
2,- (,269
3,( E,239
4,( E.,19
...,...,...
536,r. 3,11
537,( G,5
538,2-13,5
539,K -1,5


In [3]:
pattern_df = pd.read_csv("patterns.csv")
pattern_df = pattern_df.sort_values(by=["counter"], ascending=False)
pattern_df.head(20)

,pattern,counter
46,# #,11464
47,# #,6953
142,n (,3417
22,( #,3012
23,( #,2439
19,) #,2205
41,) -,1596
61,( a,1052
25,# (,1015
20,s (,989


In [10]:
img = cv2.imread("test_imgs/text_test_easy.png")

In [11]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

/home/akos/Documents/GitHub/experiments/handwritten/segmentation_with_tokens/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
pixel_values = processor(images=img, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

/home/akos/Documents/GitHub/experiments/handwritten/segmentation_with_tokens/.venv/lib/python3.7/site-packages/transformers/generation/utils.py:1357: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


If a function is continuous on the, closed interval


In [13]:
def check_patterns_in_string(text, cutoff = .3,max_char_distance=4):
    pattern_df = pd.read_csv("patterns.csv")
    patterns_as_list = pattern_df['pattern'].to_list()
    print(f"Checking patterns in '{text}'")
    
    found_patterns = []
    pattern_len = len(patterns_as_list[0])
    text_len = len(text)
    possible_pattern_count = text_len-pattern_len+1
    print(f"text_len: {text_len}, possible_pattern_count: {possible_pattern_count}")

    for pattern in patterns_as_list:
        if (pattern in text):
            found_patterns.append(pattern)
    print(f"\nfound {len(found_patterns)} patterns in the text")
    print(found_patterns)
    
    if (round(possible_pattern_count * cutoff) <= len(found_patterns)):
        print('most likely math')
    elif (round(possible_pattern_count * cutoff) <= len(found_patterns)):
        print('possibly math')
        # check distance of words
        start_indexes = []
        min_num_of_close = len(found_patterns) / 2
        num_of_close = 0
        for pattern in found_patterns:
            start_index = text.find(pattern)
            start_indexes.append(start_index)
            if len(start_indexes)!=0:
                char_distance = start_index - start_indexes[-1]
                if max_char_distance >= char_distance:
                    num_of_close = num_of_close+1
        if num_of_close >= min_num_of_close:
            print("it is math") 

    elif (round(possible_pattern_count * cutoff) > len(found_patterns)):
        print('probably text')
    else:
        print('pretty sure its text')

In [14]:
check_patterns_in_string(generated_text, .1)

['s - ', ' - (', '- ( ', ' ( E', '( E.', ' E.O', 'E.O.', '.O. ', '. ( ', ' ( (', ' (m-', '(m-2', 'm-2 ', '-2 )', '2 ) ', ' ) "', ') " ', ' " )', '" ) ', ' ) #', 's ( ', '( Es', ' ( #', '( # ', ' # (', '# ( ', ' ( "', ' "2 ', ' ) )', ') ) ', ' ( n', '( n-', '( Ex', ' Ex ', 'over', 'ver ', 'er (', 'r ( ', ' n-2', 'n-2 ', 'n ) ', ' ) -', ') - ', ' ) r', '- (m', '5 ( ', ' # #', '# # ', '( ( ', '( " ', ' " -', 'r ) ', ') # ', ' # )', '# ) ', ' # I', '# In', ' # a', ' ) (', ') ( ', ' ( T', ' ( a', 't ) ', '00 -', '0 - ', ' ( o', ' or ', 'x - ', ' Eve', 'Even', 'ven ', 'ver-', 'ange', ',000', 'been', 'een ', '9-00', '-000', '000-', 'embe', 'year', 'brie', 'rief', ' - 1', 'afte', 'fter', 'n # ', ' # y', ' you', 'you ', ' rem', 'mber', '000 ', 'tran', '1-00', '0-00', ' day', ' ( v', ' vot', 'vote', 'es -', ' - 2', '0000', ' are', 'was ', ' to ', ' 4.5', '3-40', '0.00', '.000', '" # ', ' # Y', 'y * ', 'y # ', '# I ', ' I -', ' - D', 'try ', 'ry #', '1 -0', "y't ", "'t y", 't ye', ' yet', ' # t',